<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/5_Arboles_autobalanceables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta práctica, veremos una de las implementaciones más populares de árboles autobalanceables: el árbol rojo-negro.

# Definición

Decimos que un árbol binario es un árbol rojo-negro si cumple las siguientes propiedades:

1. Cada nodo tiene asignado un color: rojo o negro.
2. La raíz del árbol es negra.
3. Los hijos de un nodo rojo son negros.
4. Para cada nodo con al menos un hijo nulo (es decir, que no tiene al menos un hijo), el número de nodos negros desde cada nodo hasta cualquiera de sus descendientes nulos es el mismo para cualquiera de sus descendientes. A este número de nodos negros se le conoce como la *altura negra*.

# Representación
## Nodos

Para representar un nodo del árbol, usamos la siguiente estructura de datos:

In [3]:
class RBNode:
    def __init__(self, val):
        self.red = False
        self.parent = None
        self.val = val
        self.left = None
        self.right = None

Con campos definidos de la siguiente manera:

* `red`: Si el nodo es rojo (`True`) o negro (`False`).
* `parent`: El padre del nodo.
* `val`: El valor del nodo.
* `left`: El hijo izquierdo.
* `right`: El hijo derecho.

## Árbol

Si bien en principio podemos utilizar tan solo los nodos para representar el árbol completo (de la forma que hicimos con BST), es más sencillo crear una segunda estructura para guardar todo el árbol. Esta tiene dos atributos:

* `nil`: El llamado "nodo sentinela" (explicado más adelante). Este nodo tiene `red=False`, y sus demás atributos `=None`.
* `root`: La raíz del árbol.

El algoritmo se vuelve mucho más sencillo si todos los nodos tienen dos hijos. Para garantizar esto, si a un nodo le falta un hijo, le asignamos como hijo el nodo sentinela de su árbol correspondiente. Por ejemplo, si tenemos el siguiente árbol:

```
      5
     /  \
    3    7
   /
  1
```

Con los nodos sentinelas (representados por `X`) se vuelve:

```
       5
     /   \
    3     7
   / \   / \
  1   X X   X
 / \
X   X
```

# Rotaciones
Las rotaciones son una operación sobre los árboles binarios que toman a un nodo como "pivote", y "rotan" a sus descendientes alrededor de él. La forma más fácil de ilustrarlas es con un diagrama:

## Rotación izquierda
En este caso, el pivote es `x`.

<img src="./img/img6.jpg" alt="Drawing" style="width: 600px;"/>

## Rotación derecha
En este caso, el pivote es `y`.

<img src="./img/img7.jpg" alt="Drawing" style="width: 600px;"/>

Podemos ver que la rotación izquierda es inversa de la derecha.

# Inserción

Los algoritmos que no alteran el árbol (búsqueda, caminata, etc.) son idénticos a los de un BST normal. Sin embargo, el algoritmo de inserción (y borrado) sufre muchos cambios, ya que al insertar el nuevo nodo, se deben de mantener las propiedades del árbol negro-rojo.

Para resolver esto, si queremos insertar un nodo nuevo `z`, le asignamos un color rojo (ya que de otra manera cambiaría la altura negra, lo cual sería difícil de resolver) y lo insertamos como si fuese un BST normal. Finalmente, llamamos una función `fix_insert` que modifica el árbol de modo que se sigan cumpliendo las propiedades.

Una vez insertado el nodo, existen dos casos posibles para el color de su padre:

1. Su padre es negro. En este caso, no tenemos que hacer nada, ya que todas las propiedades se siguen cumpliendo.
2. Su padre es rojo. Aquí, se viola la propiedad 3.

Ahora, asumiendo que ocurre el segundo caso, si el padre de `z` (llámese `p`) es un nodo izquierdo (el caso en el que es derecho es simétrico), pueden ocurrir 3 ordenamientos diferentes:

## Caso 1

<img src="./img/img1.jpg" alt="Drawing" style="width: 700px;"/>

Suponiendo que tenemos el primer caso, lo único que tenemos que hacer es "mover" el color rojo hacia arriba; de esta manera, la altura negra sigue siendo la misma.

<img src="./img/img2.jpg" alt="Drawing" style="width: 600px;"/>

Sin embargo, es posible que este desplazamiento haga que ahora el nodo padre viole la propiedad 3; por ejemplo, si el padre del nodo con valor 12 también es rojo. Para resolver esto, hacemos un nuevo ajuste, esta vez tomando al nodo padre como si fuese el nodo que insertamos.

## Casos 2 y 3

Nótese que podemos transformar el caso 2 en el caso 3 haciendo una rotación izquierda sobre el padre de `z`. Como tanto `z` como su padre son rojos, esta rotación no afectará la altura negra.

<img src="./img/img4.jpg" alt="Drawing" style="width: 400px;"/>

Una vez que tenemos el caso 3, coloreamos el padre de `z` negro, y su abuelo rojo, y luego hacemos una rotación hacia derecha sobre el abuelo de `z`.

<img src="./img/img5.jpg" alt="Drawing" style="width: 700px;"/>



In [64]:
def print_tree(node, lines=None, level=0):
    if lines is None:
        lines = []
    if node.val is not None:
        print_tree(node.left, lines, level+1)
        lines.append(f"{' ' * 4 * level} {node.val} {'r' if node.red else 'b'}")
        print_tree(node.right, lines, level+1)
    else:
        lines.append(f"{' ' * 4 * level}  x")
    return '\n'.join(lines)

class RBTree:
    def __init__(self):
        self.nil = RBNode(None)
        self.root = self.nil

    def insert(self, val):
        new_node = RBNode(val)
        new_node.parent = None
        new_node.left = self.nil
        new_node.right = self.nil
        new_node.red = True

        parent = None
        current = self.root
        while current != self.nil:
            parent = current
            if new_node.val < current.val:
                current = current.left
            elif new_node.val > current.val:
                current = current.right
            else:
                return

        new_node.parent = parent
        if parent is None:
            self.root = new_node
        elif new_node.val < parent.val:
            parent.left = new_node
        else:
            parent.right = new_node

        self.fix_insert(new_node)

    def rotate_left(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.nil:
            y.left.parent = x

        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def rotate_right(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.nil:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y
        
    def fix_insert(self, new_node):
        while new_node != self.root and new_node.parent.red:
            if new_node.parent == new_node.parent.parent.right: # si el padre es izquierdo
                u = new_node.parent.parent.left  # tío (hermano del padre)
                if u.red:
                    u.red = False
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    new_node = new_node.parent.parent
                else:
                    if new_node == new_node.parent.left: # Caso 2 
                        self.rotate_right(new_node.parent)
                        new_node = new_node.parent
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    self.rotate_left(new_node.parent.parent)
                    
            else: # si el padre es derecho
                u = new_node.parent.parent.right # tío (hermano del padre)

                if u.red:
                    u.red = False
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    new_node = new_node.parent.parent
                else:
                    if new_node == new_node.parent.right:
                        new_node = new_node.parent
                        self.rotate_left(new_node)
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    self.rotate_right(new_node.parent.parent)
        self.root.red = False

    def __repr__(self):
        return print_tree(self.root)

In [66]:
tree = RBTree()
for x in range(1, 21):
    tree.insert(x)
print(tree)

                  x
             1 b
                  x
         2 b
                  x
             3 b
                  x
     4 r
                  x
             5 b
                  x
         6 b
                  x
             7 b
                  x
 8 b
                  x
             9 b
                  x
         10 b
                  x
             11 b
                  x
     12 r
                      x
                 13 b
                      x
             14 r
                      x
                 15 b
                      x
         16 b
                      x
                 17 b
                      x
             18 r
                      x
                 19 b
                          x
                     20 r
                          x


# Ejercicios

## Ejercicio 1

Investiga e implementa el algoritmo para borrar un nodo de un árbol rojo-negro. Utiliza en la medida de lo posible las funciones existentes (i.e. rotaciones)

*Aquí va la explicación de tu algoritmo*

In [1]:
# Aquí va tu código

## Ejercicio 2

Escribe una función para convertir un árbol rojo-negro a una gráfica de `networkx`, de modo que cada nodo tenga como propiedad su color. Posteriormente, genera un árbol aleatorio, conviértelo a `networkx` y grafícalo de modo que los nodos mostrados tengan los colores correctos. Asegúrate que el árbol tenga el layout apropiado.

In [2]:
# Aquí va tu código